In [2]:
from django_for_jupyter import init_django
init_django()

In [3]:
import pandas as pd
import numpy as np
import unidecode
from django.db.models import F, Sum
from precios.models import SiteURLResults, Articulos, Unifica, Marcas, Vendedores, MarcasSistema
from members.models import DetalleLista
from precios.pi_get import fixMarca

In [3]:
### Settings
set_lower_to_nombres = True

In [4]:
## elimino todas registros anteriores
DetalleLista.objects.all().delete()
Vendedores.objects.all().delete()
Articulos.objects.all().delete()
Unifica.objects.all().update(contador=0)
# Unifica.objects.filter(automatico=True).delete()

1294

In [7]:
## Elimino todas las marcas que puedo
lista_marcas = Marcas.objects.filter(es_marca=True).all()
a = 0
for marca in lista_marcas:
    try:
        marca.delete()
    except Exception as e:
        a = a + 1

print(f'No se borran {a} marcas')

No se borran 499 marcas


In [8]:
## Se agregan mas MarcasSistema
lista_marcas = MarcasSistema.objects.all()
a = 0
for marca in lista_marcas:
    marca_limpia = unidecode.unidecode(marca.nombre)
    if not Marcas.objects.filter(nombre=marca.nombre).exists():
        Marcas.objects.update_or_create(
            nombre=marca.nombre,
            slug=marca_limpia,
            es_marca=True
        )
        a = a + 1
print(f'Se Agregan {a} marcas')

Se Agregan 453 marcas


In [9]:
# Obtener todos los registros de SiteURLResults
querySiteURL = SiteURLResults.objects.exclude(precio=0, nombre='', marca=None, site__enable=True).all()

## genero marcas
df_marcas = pd.DataFrame(list(querySiteURL.values('marca')))
df_marcas['marca'] = df_marcas['marca'].str.lower()
df_marcas = df_marcas.sort_values(by='marca')
df_marcas = df_marcas.drop_duplicates(keep='first')

a = 0
### Se crean todas las marcas necesarias
for marca in df_marcas['marca']:
    if marca:
        fixed_marca = fixMarca(marca)
        marca_limpia = unidecode.unidecode(fixed_marca)

        if not Marcas.objects.filter(nombre__iexact=fixed_marca).exists():
            marca_limpia, created = Marcas.objects.update_or_create(nombre = fixed_marca, 
                                                                    slug = marca_limpia, 
                                                                    es_marca=True)
            a = a + 1

print(a)

2849


In [10]:
# Obtener todos los registros de Marcas
querymarcas = Marcas.objects.all()
df_marcas = pd.DataFrame(list(querymarcas.values('nombre')))



In [11]:
# # Obtener todos los registros de Unificas
# queryunificas = Unifica.objects.all()
# df_unificas = pd.DataFrame(list(queryunificas.values('si_marca__nombre',
#                                                      'si_nombre',
#                                                      'si_grados',
#                                                      'si_medida_cant',
#                                                      'si_unidades',
#                                                      'entonces_marca__nombre',
#                                                      'entonces_nombre',
#                                                      'entonces_grados',
#                                                      'entonces_medida_cant',
#                                                      'entonces_unidades',
#                                                      'automatico'
#                                                     )))

# Convertir a un DataFrame de Pandas
df_urls = pd.DataFrame(list(querySiteURL.values('site', 'nombre','marca','medida_cant', 'unidades', 'idproducto', 'precio')))

In [12]:
df_urls['nombre'] = df_urls['nombre'].str.strip()
df_urls['marca'] = df_urls['marca'].str.strip()
df_urls['unidades'] = df_urls['unidades'].str.strip()
df_urls['idproducto'] = df_urls['idproducto'].str.strip()

In [13]:
df_marcas['nombre'] = df_marcas['nombre'].str.strip()
df_urls['marca'] = df_urls['marca'].str.strip()
df_urls['marca'] = df_urls['marca'].str.lower()

In [14]:
df_urls.dropna(inplace=True)
        

### generar lista aquellos sin marca


In [15]:
df_sin_marca = df_urls[df_urls['marca'] == '']
df_con_marca = df_urls[df_urls['marca'] != '']


## Lista de marcas no existentes

In [16]:
lista_marcas_no_existen = df_con_marca[~df_con_marca['marca'].isin(df_marcas['nombre'])]
lista_marcas_no_existen = lista_marcas_no_existen.drop(columns=['site', 'nombre', 'medida_cant', 'unidades', 'precio', 'idproducto'])
lista_marcas_no_existen = lista_marcas_no_existen.sort_values(by='marca')
lista_marcas_no_existen = lista_marcas_no_existen.drop_duplicates(keep='first')

In [17]:
existen, no_existen = 0,0
### Se crean todas las marcas necesarias
for marcas_no_existe in lista_marcas_no_existen['marca']:
    
    marca_limpia = unidecode.unidecode(marcas_no_existe)
    
    fixed_marca = fixMarca(marca_limpia)
    
    if not Marcas.objects.filter(nombre__iexact=fixed_marca).exists():
        marca_limpia, created = Marcas.objects.update_or_create(nombre = fixed_marca)
        
        
#     if not Marcas.objects.filter(nombre__iexact=marca_limpia).exists():
#         marca_limpia, created = Marcas.objects.update_or_create(nombre = marca_limpia)
        
    if not Marcas.objects.filter(nombre__iexact=marcas_no_existe).exists():
        marcas_no_existe, created = Marcas.objects.update_or_create(nombre = marcas_no_existe)

    
for marcas_no_existe in lista_marcas_no_existen['marca']:    
    ## Objetos
    marca_limpia = unidecode.unidecode(marcas_no_existe)
    
    fixed_marca = fixMarca(marca_limpia)

    nueva_marca_obj = Marcas.objects.filter(nombre__iexact = marcas_no_existe).get()
    fixed_marca_obj = Marcas.objects.filter(nombre__iexact = fixed_marca).get()
    if nueva_marca_obj != fixed_marca_obj:
        Unifica.objects.update_or_create(
                si_marca=nueva_marca_obj,
                entonces_marca=fixed_marca_obj,
                contador=0,
                automatico=True
           )
    



## Campo marca vacio, la marca en el nombre


In [18]:
listamarcas = list(Marcas.objects.filter(es_marca=True).all())
urlsinMarca = SiteURLResults.objects.filter(marca='').all()
registro = 0
marcas_doble = 0
una_marca  = 0
for url in urlsinMarca:
    registro = registro  + 1
    nombre = url.nombre.lower()
    num_marcas = 0
    arr_marcas = []
    for posible_marrca in listamarcas:
        busca_marca = posible_marrca.nombre.lower()
        
        if ( ' ' + busca_marca +' ' in nombre ) or \
            ( '.' + busca_marca +' ' in nombre ):
            num_marcas = num_marcas + 1
            arr_marcas.append(busca_marca)
            ultima_marca = busca_marca
            
    if num_marcas == 1:
        print(f'busca_marca= {ultima_marca}  en {url.nombre} ')
        # url.marca = ultima_marca
        # url.nombre = nombre.replace(ultima_marca, '')
        # url.save()
        una_marca = una_marca + 1
        
    else:
        marcas_doble = marcas_doble + 1

print(f'registros= {registro} marcas_doble={marcas_doble}  una_marca={una_marca}')

registros= 35286 marcas_doble=735  una_marca=34551


In [19]:
# Obtener todos los registros de SiteURLResults
querySiteURL = SiteURLResults.objects.exclude(precio=0, nombre='', marca='').all()

# Convertir a un DataFrame de Pandas
df_urls = pd.DataFrame(list(querySiteURL.values('pk', 'site', 'nombre','marca','medida_cant', 'unidades', 'idproducto', 'precio')))
df_urls = df_urls.replace(r'^\s*$', value=None, regex=True)

# reemplazar el carácter acentuado por el carácter sin acento en la columna de nombre
df_urls['nombre'] = df_urls['nombre'].str.replace("´", "'")

df_urls

,pk,site,nombre,marca,medida_cant,unidades,idproducto,precio
0,593250,4,None,zuko,0.0,None,None,2530
1,587011,4,None,zuko,0.0,None,None,1690
2,675848,4,None,zuko,0.0,None,None,7990
3,589301,4,None,zuko,0.0,None,None,7290
4,588521,4,None,zuko,0.0,None,None,3750
...,...,...,...,...,...,...,...,...
92217,583544,4,None,Oral B,0.0,None,766331,3530
92218,590294,4,None,Quillayes,0.0,None,276563,850
92219,588523,4,None,Casillero del Diablo,0.0,None,709647,7750
92220,582975,4,None,Loreal,0.0,None,621339,3190


In [ ]:
cuenta = 0
# for i, row in df_urls.iterrows():
#     cuenta = cuenta + 1

#     url = SiteURLResults.objects.get(pk=row['pk'])
#     url.nombre      = row['nombre']
#     url.marca       = row['marca']
#     url.medida_cant = row['medida_cant']
#     url.unidades    = row['unidades']
#     url.idproducto  = row['idproducto']
#     url.save()
    
updated_urls = []
for i, row in df_urls.iterrows():
    cuenta = cuenta + 1
    url = SiteURLResults(
        pk=row['pk'],
        nombre=row['nombre'],
        marca=row['marca'],
        medida_cant=row['medida_cant'],
        unidades=row['unidades'],
        idproducto=row['idproducto']
    )
    updated_urls.append(url)

SiteURLResults.objects.bulk_update(
    updated_urls,
    ['nombre', 'marca',  'medida_cant', 'unidades', 'idproducto']
)

In [ ]:
print(cuenta)

In [ ]:
UPDATE precios_articulos
SET ean_13 = REPLACE(ean_13, 'sku:', '')
WHERE ean_13 LIKE '%sku:%';

UPDATE precios_articulos
SET ean_13 = REPLACE(ean_13, '\t\t\t\t\n\n', '')
WHERE ean_13 LIKE '%\t\t\t\t\n\n%';
UPDATE precios_articulos
SET ean_13 = REPLACE(ean_13, '\t', '')
WHERE ean_13 LIKE '%\t%';
UPDATE precios_articulos
SET ean_13 = REPLACE(ean_13, '\n', '')
WHERE ean_13 LIKE '%\n%';

UPDATE precios_articulos SET ean_13 = REPLACE(ean_13, ' ', '');



UPDATE precios_siteurlresults
SET idproducto = REPLACE(idproducto, 'sku:', '')
WHERE idproducto LIKE '%sku:%';

UPDATE precios_siteurlresults
SET idproducto = REPLACE(idproducto, '\t\t\t\t\n\n', '')
WHERE idproducto LIKE '%\t\t\t\t\n\n%';

UPDATE precios_siteurlresults
SET idproducto = REPLACE(idproducto, '\t', '')
WHERE idproducto LIKE '%\t%';

UPDATE precios_siteurlresults
SET idproducto = REPLACE(idproducto, '\n', '')
WHERE idproducto LIKE '%\n%';

UPDATE precios_siteurlresults SET idproducto = REPLACE(idproducto, ' ', '');